### Matching Maritime & Coastguard Agency recruitment & placement agencies to companies house data

In [1]:
import requests
import io
import pandas as pd
import numpy as np
import probablepeople as pp
from bs4 import BeautifulSoup

In [2]:
# Function to strip stopwords from company names

def strip_stopwords(raw_name):    
    company_stopwords = { 'LIMITED', 'LTD', 'SERVICES', 'COMPANY', 'GROUP', 'PROPERTIES', 'CONSULTING', 
        'HOLDINGS', 'UK', 'TRADING', 'LTD.' }
    return(' '.join([raw_name_part for raw_name_part in raw_name.split() if raw_name_part not in company_stopwords]))

# Function to extract postcode as last 2 substrings in address
# To Fix: Not perfect if postcode not last entry in address

def extract_postcode(address):
    return ' '.join(address.split()[-2:])

In [3]:
# Maritime and Coastguard Agency - List of approved recruitment and placement agencies

url = "https://www.gov.uk/government/publications/recruitment-and-placement-agencies-approved-by-the-mca"

In [4]:
# Download the csv file, extract name and drop erroroneous last row with all nulls

with requests.Session() as req:
    r = req.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    targets = [f"{item['href']}" for item in soup.select(
        "a[href$='.csv']")]
    for target in targets:
        response = req.get(target)    
        mdf = pd.read_csv(io.BytesIO(response.content))
        mdf = mdf.dropna(how='all')

In [5]:
# Extract Postcode and Cleanse Name Column of Stopwords
# Create unique index column needed by Splink from datafram index

mdf['ADDRESS & CONTACT DETAILS']=mdf['ADDRESS & CONTACT DETAILS'].astype('string')
mdf['Postcode'] = mdf.apply(lambda row: extract_postcode(row['ADDRESS & CONTACT DETAILS']), axis=1)
mdf['Location'] = mdf['LOCATION'].str.upper()
mdf['CompanyName'] = mdf['COMPANY'].str.upper()
mdf['CompanyName'].replace('', np.nan, inplace=True)
mdf['CompanyName'] = mdf['CompanyName'].astype(str)
mdf['CompanyName'] = mdf.apply(lambda row: strip_stopwords(row['CompanyName']), axis=1)
mdf['unique_id'] = mdf.index

In [6]:
# Subset down to required columns
# Number of agencies to match

mdf = mdf[['Postcode','CompanyName','Location','unique_id']]
len(mdf)

103

In [7]:
# Read basic company details

cdf = pd.read_csv('basic_slim.csv')
cdf = cdf.rename(columns={"RegAddress.PostCode": "Postcode", 'RegAddress.PostTown': 'Location'})
cdf['CompanyName'] = cdf.apply(lambda row: strip_stopwords(row['CompanyName']), axis=1)
cdf['CompanyName'].replace('', np.nan, inplace=True)
cdf['unique_id'] = cdf.index

In [8]:
# Subset down to required columns
# Number of companies to match against

cdf = cdf[['Postcode','CompanyName','Location','unique_id']]
len(cdf)

5184795

In [9]:
# Number of exact matches

exact = mdf.merge(cdf,left_on=['Postcode','CompanyName'], right_on=['Postcode','CompanyName'],
          suffixes=('_left', '_right'))
len(exact)

50

In [10]:
import recordlinkage

In [11]:
# Block on postcode
# Number of possible links

indexer = recordlinkage.Index()
indexer.block("Postcode")
candidate_links = indexer.index(cdf, mdf)
len(candidate_links)

150220

In [12]:
# Number of matches on postcode and name

compare_cl = recordlinkage.Compare()
compare_cl.string("CompanyName", "CompanyName", method='jarowinkler',threshold=0.85)
compare_cl.exact("Postcode","Postcode")
features = compare_cl.compute(candidate_links, cdf, mdf)

# Name exact or approx match and postcode matches
matches = features[(features[0]==1) & (features[1]==1)]
len(matches)

121

In [13]:
# Rename indexes to allow join

matches.index.names = ['cdf','mdf']
cdf.index.names= ['cdf']
mdf.index.names= ['mdf']

# Lookup both names
matches = matches.join(cdf, how='inner')
matches = matches.join(mdf, how='inner', rsuffix='_mdf')

# Select those with only approx match not exact match
approx = matches[matches['CompanyName']!=matches['CompanyName_mdf']]
len(approx)

71

In [14]:
# List of those companies matched at least once
found = matches.index.unique(level='mdf')
len(found)

59

In [15]:
# Select those companies not found 
notfound = mdf.loc[(mdf.index.isin(found, level='mdf') == False)]
len(notfound)

44

In [16]:
# Postcode matches but name doesn't
postmatches = features[(features[0]!=1) & (features[1]==1)]
postmatches.index.names = ['cdf','mdf']

postmatches = postmatches.join(cdf, how='outer')
postmatches = postmatches.join(mdf, how='inner', rsuffix='_mdf')
len(postmatches)

150099

In [17]:
import splink

In [18]:
from splink.duckdb.duckdb_linker import DuckDBLinker
from splink.duckdb import duckdb_comparison_library as cl
settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        "l.Postcode = r.Postcode",
    ],
    "comparisons": [
        cl.jaro_winkler_at_thresholds("CompanyName",[0.9]),
 #       cl.exact_match("Location"),
    ],    
    "retain_intermediate_calculation_columns" : True,
    "retain_matching_columns" : True
}

In [19]:
# Use 1e7 targets rows to ensure u estimation

linker = DuckDBLinker([mdf, cdf], settings, input_table_aliases=["mdf", "cdf"])
linker.estimate_u_using_random_sampling(target_rows=1e7)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - CompanyName (no m values are trained).


In [20]:
linker.estimate_parameters_using_expectation_maximisation("l.Postcode = r.Postcode")


----- Starting EM training session -----

Estimating the m probabilities of the model by blocking on:
l.Postcode = r.Postcode

Parameter estimates will be made for the following comparison(s):
    - CompanyName

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 

Iteration 1: Largest change in params was 0.0637 in the m_probability of CompanyName, level `jaro_winkler_similarity >= 0.9`
Iteration 2: Largest change in params was 0.206 in the m_probability of CompanyName, level `jaro_winkler_similarity >= 0.9`
Iteration 3: Largest change in params was 0.0513 in the m_probability of CompanyName, level `jaro_winkler_similarity >= 0.9`
Iteration 4: Largest change in params was 0.00388 in the m_probability of CompanyName, level `jaro_winkler_similarity >= 0.9`
Iteration 5: Largest change in params was 0.000251 in the m_probability of CompanyName, level `jaro_winkler_similarity >= 0.9`
Iteration 6: Largest change in params was 1.6e-0

<EMTrainingSession, blocking on l.Postcode = r.Postcode, deactivating comparisons >

In [41]:
df_splink = linker.predict(threshold_match_probability=0.7).as_pandas_dataframe()
len(df_splink)

79

In [42]:
df_nonexact = df_splink[(df_splink['CompanyName_l']!=df_splink['CompanyName_r'])]
df_nonexact

,match_weight,match_probability,source_dataset_l,unique_id_l,source_dataset_r,unique_id_r,CompanyName_l,CompanyName_r,gamma_CompanyName,bf_CompanyName,Postcode_l,Postcode_r
5,1.509362,0.740046,cdf,4648491,mdf,91,THE SHIP'S PHOTOGRAPHER (SERVICES),THE SHIPS PHOTOGRAPHER,1,28465.559992,SO14 3LP,SO14 3LP
8,1.509362,0.740046,cdf,2391973,mdf,45,JAMES FISHER (ABERDEEN),JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
9,1.509362,0.740046,cdf,2391977,mdf,45,JAMES FISHER ANGOLA,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
10,1.509362,0.740046,cdf,2391980,mdf,45,JAMES FISHER EVERARD,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
11,1.509362,0.740046,cdf,2391981,mdf,45,JAMES FISHER,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
13,1.509362,0.740046,cdf,2391984,mdf,45,JAMES FISHER MFE,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
14,1.509362,0.740046,cdf,2391985,mdf,45,JAMES FISHER MIMIC,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
15,1.509362,0.740046,cdf,2391986,mdf,45,JAMES FISHER NUCLEAR,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
16,1.509362,0.740046,cdf,2391991,mdf,45,JAMES FISHER RUMIC,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR
17,1.509362,0.740046,cdf,2391993,mdf,45,JAMES FISHER TANKSHIPS,JAMES FISHER MARINE,1,28465.559992,LA14 1HR,LA14 1HR


In [43]:
df_nonexact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29 entries, 5 to 76
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   match_weight       29 non-null     float64
 1   match_probability  29 non-null     float64
 2   source_dataset_l   29 non-null     object 
 3   unique_id_l        29 non-null     int64  
 4   source_dataset_r   29 non-null     object 
 5   unique_id_r        29 non-null     int64  
 6   CompanyName_l      29 non-null     object 
 7   CompanyName_r      29 non-null     object 
 8   gamma_CompanyName  29 non-null     int32  
 9   bf_CompanyName     29 non-null     float64
 10  Postcode_l         29 non-null     object 
 11  Postcode_r         29 non-null     object 
dtypes: float64(3), int32(1), int64(2), object(6)
memory usage: 2.8+ KB


In [45]:
linker.save_settings_to_json("CompanyMatchSplinkSettings.json", overwrite=True)

{'link_type': 'link_only',
 'blocking_rules_to_generate_predictions': ['l.Postcode = r.Postcode'],
 'comparisons': [{'output_column_name': 'CompanyName',
   'comparison_levels': [{'sql_condition': '"CompanyName_l" IS NULL OR "CompanyName_r" IS NULL',
     'label_for_charts': 'Null',
     'is_null_level': True},
    {'sql_condition': '"CompanyName_l" = "CompanyName_r"',
     'label_for_charts': 'Exact match',
     'm_probability': 0.6448052118710551,
     'u_probability': 1.8790723020045003e-07},
    {'sql_condition': 'jaro_winkler_similarity("CompanyName_l", "CompanyName_r") >= 0.9',
     'label_for_charts': 'jaro_winkler_similarity >= 0.9',
     'm_probability': 0.3503519369852797,
     'u_probability': 1.2307923578129478e-05},
    {'sql_condition': 'ELSE',
     'label_for_charts': 'All other comparisons',
     'm_probability': 0.004842851143665266,
     'u_probability': 0.9999875041691917}],
   'comparison_description': 'Exact match vs. jaro_winkler_similarity at threshold 0.9 vs. an